In [6]:
import sys
import os
IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR)



In [7]:
import imatools.vtktools as vtktools
import imatools.ioutils as iou

print(iou.chooseplatform())

macOS


In [8]:

# Locations of hard drive based on platform
dirdic={'macOS'   : '/Volumes/sandisk', 
        'Linux'   : '/media/jsl19/sandisk' , 
        'Windows' : 'D:/'}

p2f = iou.fullfile(dirdic[iou.chooseplatform()], '09-dnav_vs_inav/umc/local')
N = os.listdir(p2f)
xnav = lambda x : 'LGE_' + x + 'NAV'


# Project iNAV values on dNAV mesh

In [9]:
n = 1  # 0..22 (23 cases)
case_path = iou.fullfile(p2f, str(N[n]))
inav = iou.fullfile(case_path, xnav('i'), 'OUTPUT')
dnav = iou.fullfile(case_path, xnav('d'), 'OUTPUT')

dpath = iou.searchFileByType(dnav, 'Normalised', 'vtk')[0]
ipath = iou.searchFileByType(inav, 'Normalised', 'vtk')[0]

msh_d = vtktools.readVtk(dpath) # source
msh_i = vtktools.readVtk(ipath) # target

msh_i_on_d = vtktools.projectCellData(msh_source=msh_d, msh_target=msh_i)

vtktools.writeVtk(msh_i_on_d, dnav, "Normalised_iNav_on_dNav")
        

# Compare meshes and projections 
We compare meshes

+ `msh_d = ` dNAV , and 
+ `msh_i_on_d = ` iNAV 

Both have been loaded from the `Normalised` version, so thresholds are simple, e.g `1.2, 1.3`, etc


In [10]:
## create thresholded masks of each mesh
import vtk 

print(case_path)

d_thres = 1.2 
iod_thres = 1.15 # try different thresholds 

omsh = vtk.vtkPolyData()
omsh.DeepCopy(msh_d)
o_scalar = vtk.vtkFloatArray()
o_scalar.SetNumberOfComponents(1)

iod_scalar = msh_i_on_d.GetCellData().GetScalars()
d_scalar = msh_d.GetCellData().GetScalars()

for ix in range(msh_d.GetNumberOfCells()) : 
    value_assigned = 0

    if (d_scalar.GetTuple1(ix) == 0) : 
        value_assigned = -1

    else :  
        if (d_scalar.GetTuple1(ix) >= d_thres) : 
            value_assigned += 1

        if (iod_scalar.GetTuple1(ix) >= iod_thres) : 
            value_assigned += 2
    
    o_scalar.InsertNextTuple1(value_assigned)

omsh.GetCellData().SetScalars(o_scalar)
vtktools.writeVtk(omsh, case_path, "overlap")


/Volumes/sandisk/09-dnav_vs_inav/umc/local/5


In [13]:
d_thres = 1.2
iod_thres = 1.2  # try different thresholds

omsh, Counts = vtktools.fibrosisOverlapCell(msh_d, msh_i_on_d, d_thres, iod_thres, name0='dnav', name1='inav') 

tp = Counts['overlap']
tn = Counts['none']
fp = Counts['inav']
fn = Counts['dnav']

perf = iou.performanceMetrics(tp, tn, fp, fn)
print(perf)


{'jaccard': 0.2368011323425336, 'precision': 0.38572087304027053, 'recall': 0.38016967126193, 'accuracy': 0.3557688478234896}
